In [1]:
using Compat
using Docile
using Iterators
using Pipe

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [2]:
push!(LOAD_PATH, map(x->"../"*x, filter(fn-> !(contains(fn,".")),readdir("..")))...)


11-element Array{Union(ASCIIString,UTF8String),1}:
 "/root/buildFromSource/julia/usr/local/share/julia/site/v0.3"
 "/root/buildFromSource/julia/usr/share/julia/site/v0.3"      
 "../Corpus"                                                  
 "../doc2vec"                                                 
 "../Models"                                                  
 "../Optimisation"                                            
 "../recursive_embeddings"                                    
 "../summaristation"                                          
 "../tools"                                                   
 "../util"                                                    
 "../word-embedding3"                                         

In [3]:
using WordEmbeddings

In [4]:
training = open("../Corpus/serialised/opinosis_train_dev_plain.jsz","r") do fs
    deserialize(fs)
end
@pz training

training		Array{Array{String,1},1}	(6097,)


In [5]:
function pad{S<:String}(sent::Vector{S}, padded_length, pad_word="*STARTPAD*")
    if length(sent) <= padded_length
        ret = fill(pad_word,padded_length)
        ret[end-length(sent)+1:end] = sent
        ret
    else
        sent
    end
end

pad (generic function with 2 methods)

In [27]:

import WordEmbeddings.WE
function WE(N::DataType,S::DataType, embedding_width::Int)
    L=Array(N,(embedding_width,0))
    word_index=Dict{S,Int}()
    indexed_words=S[]
    WE(L,word_index,indexed_words)
end



@doc "Gets the word index, or creates one if it doesn't already exist" ->
function get_word_index!{N,S, S2}(we::WE{N,S}, word::S2, word_varience = 0.01)
    @assert S2<:S
    if (word in keys(we.word_index))
        we.word_index[word]
    else
        index = length(we.indexed_words)+1
        we.word_index[word]=index
        push!(we.indexed_words,word)
        
        embedding = convert(Vector{N},word_varience.*randn(size(we.L,1)))
        we.L = hcat(we.L,embedding)
        index
    end
end

function add_all_words!{N,S}(we::WE{N,S}, words::Vector{S}, word_varience=0.01)
    for word in words
        get_word_index!(we, word, word_varience)
    end
    we
end
function add_all_words!{N,S}(we::WE{N,S}, paras::Vector{Vector{S}}, word_varience=0.01)
    for para in paras
        add_all_words!(we, para, word_varience)
    end
    we
end

add_all_words! (generic function with 4 methods)

In [7]:
type PVDM{N<:Number, S<:AbstractString}
    we::WE
    
    pe::WE #use a word embedder for Paragraphs too
    
    
    W::AbstractMatrix{N}
    b::AbstractVector{N}

    window_length::Int
    varience::N
end

function PVDM{N,S}(we::WE{N,S}, window_length::Int, varience=0.001)
    
    emb_width,n_words = size(we.L)
    concat_layer_width = emb_width*(window_length+1)
    const W = convert(Matrix{N}, varience*randn(n_words,concat_layer_width))
    const b = convert(Vector{N}, varience*randn(n_words))
    
    pe = WE(N,Vector{S},emb_width)
    
    PVDM{N,S}( we, pe, W,b, window_length, varience)
end
    
    

PVDM{N<:Number,S<:String} (constructor with 2 methods)

In [32]:
we_outer = WE(Float32,String, 200)
add_all_words!(we_outer, training)
pvdm = PVDM(we_outer, 8);
we_outer=0

0

In [37]:
paddded_training = map(ws->pad(ws, pvdm.window_length+1), training)

6097-element Array{Array{T,N},1}:
 String["being","able","to","change","the","*UNKNOWN*","sizes","is","awesome","!"]                                                                                                                   
 String["for","whatever","reason",",","*UNKNOWN*","decided","to","make","the","*UNKNOWN*","on","the","home","screen","than","on","the","*UNKNOWN*","."]                                              
 String["i","found","myself","constantly","changing","the","angle","of","the","body"  …  "up","and","down","and","the","distance","away","from","me","."]                                            
 String["i","was","an","avid","reader","but","increasing","age","has","made"  …  "and","very","light","weight","has","made","reading","fun","again","."]                                             
 String["what","'s","more",",","it","'s","easy","to","change","*UNKNOWN*","size","."]                                                                                         

In [ ]:
function get_training_cases(window_len)